In [2]:
from pathlib import Path
import os
from neuralhydrology.evaluation import metrics, get_tester
from neuralhydrology.utils.config import Config
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas as pd
import yaml
import warnings
warnings.simplefilter("ignore", FutureWarning)
import plotly.graph_objects as go


In [3]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go

Allereerst bepalen wij bij welke epoch de modellen de hoogste NSE hebben, daarna plotten we de scores

In [134]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import os 
def extract_tensorboard_scalars(logdir):
    event_acc = EventAccumulator(logdir)
    event_acc.Reload()  # Load the TensorBoard logs

    scalars = {}
    for tag in event_acc.Tags()['scalars']:  # Get all scalar tags
        scalars[tag] = [(e.step, e.value) for e in event_acc.Scalars(tag)]
    
    return scalars

Hieronder base_folder aanpassen naar je eigen mapje.

In [135]:
base_folder = r'C:\Users\christiaan.wewer\Documents\projecten\Z0264 - Neural Hydrology - Onderzoek Afvoerverwachtingen HDSR o.b.v. AI\Gegevens\Bewerking\Scripts\visualisatie_runs'

In [136]:
r'C:\Users\christiaan.wewer\Documents\projecten\Z0264 - Neural Hydrology - Onderzoek Afvoerverwachtingen HDSR o.b.v. AI\Gegevens\Bewerking\Scripts\visualisatie_runs\runs\development_run_1303_145926'

'C:\\Users\\christiaan.wewer\\Documents\\projecten\\Z0264 - Neural Hydrology - Onderzoek Afvoerverwachtingen HDSR o.b.v. AI\\Gegevens\\Bewerking\\Scripts\\visualisatie_runs\\runs\\development_run_1303_145926'

In [137]:
runs_folder = r'runs_totaal'
run_names = ['development_run_1303_145926', 'development_run_1303_180110'] # later in de grafieken ook wel model 1 en 2 genoemd, respectivelijk

In [168]:
# check which folders are in the runs folder
run_names = [f for f in os.listdir(os.path.join(base_folder, runs_folder)) if os.path.isdir(os.path.join(base_folder, runs_folder, f))]
# for now we exclude run 1 and 2 bc they are slightly different in terms of setup
run_names = run_names[2:]

In [174]:
# get NSE curves of validation data
epoch_nr_max_avg_nse = []
max_avg_nse_value = []
epoch_nr_max_nse_1d = []
epoch_nr_max_nse_1h = []

for r in run_names:
    folder = f'{base_folder}\{runs_folder}\{r}'
    data = extract_tensorboard_scalars(folder)
    # we pakken de mediaan want doordat er paar gebieden niet zo goede resultaten hebben, kunnen die het gemiddelde naar beneden trekken
    nse_vals_1d = np.array([score for epoch, score in data['valid/median_nse_1d']]) 
    nse_vals_1h = np.array([score for epoch, score in data['valid/median_nse_1h']])
    epoch_nr_max_nse_1d.append((np.argmax(nse_vals_1d)+1)*5)
    epoch_nr_max_nse_1h.append((np.argmax(nse_vals_1h)+1)*5)
    nse_vals_avg = (nse_vals_1d + nse_vals_1h) / 2
    max_avg_nse_epoch = (np.argmax(nse_vals_avg)+1)*5 # +1 because of 0 indexing, * 5 because we save per 5 epochs
    epoch_nr_max_avg_nse.append(max_avg_nse_epoch)
    max_avg_nse_value.append(np.max(nse_vals_avg))


    # print(f'De maximale NSE van model {r} is bij epoch {max_nse_epoch}')


In [175]:
pd.DataFrame({'Epoch With Max NSE-1D':epoch_nr_max_nse_1d,\
                'Epoch With Max NSE-1H':epoch_nr_max_nse_1h,\
                'Epoch With Max AVG NSE':epoch_nr_max_avg_nse}, index=run_names)

,Epoch With Max NSE-1D,Epoch With Max NSE-1H,Epoch With Max AVG NSE
development_run_1803_153901,15,50,15
development_run_1903_083038,55,20,55
development_run_1903_105053,30,25,25
development_run_1903_131019,40,40,40
development_run_1903_153018,75,25,25
development_run_1903_175009,75,75,75
development_run_1903_201011,55,70,75
development_run_2003_095615,75,65,75
development_run_2003_122328,60,55,55
development_run_2003_145028,75,60,60


In [153]:
# Cell that brings back the backup files, these in each run_names folder in the folder backup and need to be moved one directory UP
# for r in run_names:
#     folder = f'{base_folder}\{runs_folder}\{r}'
#     backup_folder = f'{folder}\\backup'
#     for file in os.listdir(backup_folder):
#         os.rename(f'{backup_folder}\{file}', f'{folder}\{file}')

In [162]:
# per folder in run_names I want to make a backup folder if it does not exist yet. 
# Then I want to copy all the files that end with epochXXX.pt to the backup folder, 
# where XXX are all numbers that are higher than the corresponding max_nse_per_epoch
for r, max_nse_epoch in zip(run_names, epoch_nr_max_avg_nse):
    folder = f'{base_folder}\{runs_folder}\{r}'
    backup_folder = f'{base_folder}\{runs_folder}\{r}\\backup'
    if not os.path.exists(backup_folder):
        os.makedirs(backup_folder)
    for file in os.listdir(folder):
        if file.endswith('.pt'):
            epoch = int(file.split('epoch')[-1].split('.')[0])
            if epoch > max_nse_epoch:
                print(file, epoch)

                # CUT file to backup folder
                os.rename(f'{folder}\{file}', f'{backup_folder}\{file}')


model_epoch020.pt 20
model_epoch025.pt 25
model_epoch030.pt 30
model_epoch035.pt 35
model_epoch040.pt 40
model_epoch045.pt 45
model_epoch050.pt 50
model_epoch055.pt 55
model_epoch060.pt 60
model_epoch065.pt 65
model_epoch070.pt 70
model_epoch075.pt 75
model_epoch080.pt 80
optimizer_state_epoch020.pt 20
optimizer_state_epoch025.pt 25
optimizer_state_epoch030.pt 30
optimizer_state_epoch035.pt 35
optimizer_state_epoch040.pt 40
optimizer_state_epoch045.pt 45
optimizer_state_epoch050.pt 50
optimizer_state_epoch055.pt 55
optimizer_state_epoch060.pt 60
optimizer_state_epoch065.pt 65
optimizer_state_epoch070.pt 70
optimizer_state_epoch075.pt 75
optimizer_state_epoch080.pt 80
model_epoch060.pt 60
model_epoch065.pt 65
model_epoch070.pt 70
model_epoch075.pt 75
optimizer_state_epoch060.pt 60
optimizer_state_epoch065.pt 65
optimizer_state_epoch070.pt 70
optimizer_state_epoch075.pt 75
model_epoch030.pt 30
model_epoch035.pt 35
model_epoch040.pt 40
model_epoch045.pt 45
model_epoch050.pt 50
model_epoch

In [167]:
folder

'C:\\Users\\christiaan.wewer\\Documents\\projecten\\Z0264 - Neural Hydrology - Onderzoek Afvoerverwachtingen HDSR o.b.v. AI\\Gegevens\\Bewerking\\Scripts\\visualisatie_runs\\runs_totaal\\development_run_2103_130022'

In [176]:
# read the configs of the models 
hiddens_dynamics_embedding = []
hiddens_static_embedding = []
learning_rate = []
for r in run_names:
    folder = f'{base_folder}\{runs_folder}\{r}'
    with open(f'{folder}\\config.yml') as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
    if 'dynamics_embedding' in config:
        hiddens_dynamics_embedding.append(str(config['dynamics_embedding']['hiddens']))
    else:
        hiddens_dynamics_embedding.append('not')

    hiddens_static_embedding.append(str(config['statics_embedding']['hiddens']))
    learning_rate.append(config['learning_rate'][0])
    
    # config

In [177]:
run_names

['development_run_1803_153901',
 'development_run_1903_083038',
 'development_run_1903_105053',
 'development_run_1903_131019',
 'development_run_1903_153018',
 'development_run_1903_175009',
 'development_run_1903_201011',
 'development_run_2003_095615',
 'development_run_2003_122328',
 'development_run_2003_145028',
 'development_run_2003_171738',
 'development_run_2003_194450',
 'development_run_2003_221220',
 'development_run_2103_003954',
 'development_run_2103_030724',
 'development_run_2103_053442',
 'development_run_2103_080210',
 'development_run_2103_103232']

In [178]:
df_results = pd.DataFrame({'hiddens_dynamics_embedding':hiddens_dynamics_embedding,\
                            'hiddens_static_embedding':hiddens_static_embedding,\
                            'learning_rate':learning_rate,\
                            'max_avg_nse_value':max_avg_nse_value}, index=run_names)

In [194]:
df_results

,hiddens_dynamics_embedding,hiddens_static_embedding,learning_rate,max_avg_nse_value
development_run_1803_153901,[32],[128],0.0005,0.452472
development_run_1903_083038,"[64, 64]",[32],0.0005,0.467783
development_run_1903_105053,"[64, 64]","[64, 64]",0.0005,0.478346
development_run_1903_131019,[32],[64],0.0005,0.491851
development_run_1903_153018,[128],"[64, 64]",0.0005,0.472811
development_run_1903_175009,"[32, 32]","[64, 64]",5e-05,0.440265
development_run_1903_201011,not,[32],5e-05,0.369127
development_run_2003_095615,"[32, 32]",[64],0.0001,0.460981
development_run_2003_122328,"[32, 32]",[128],0.0001,0.437849
development_run_2003_145028,[32],[128],1e-05,0.179698


In [195]:
df_results.sort_values('max_avg_nse_value', ascending=False)

,hiddens_dynamics_embedding,hiddens_static_embedding,learning_rate,max_avg_nse_value
development_run_2103_030724,[64],"[64, 64]",0.0005,0.520296
development_run_2003_221220,[128],"[64, 64]",0.0005,0.519299
development_run_2103_103232,[64],"[64, 64]",0.0005,0.499427
development_run_1903_131019,[32],[64],0.0005,0.491851
development_run_2003_194450,[128],"[32, 32]",0.001,0.489615
development_run_1903_105053,"[64, 64]","[64, 64]",0.0005,0.478346
development_run_2103_003954,[128],"[64, 64]",0.0005,0.476231
development_run_1903_153018,[128],"[64, 64]",0.0005,0.472811
development_run_2103_053442,[64],"[64, 64]",0.001,0.471439
development_run_1903_083038,"[64, 64]",[32],0.0005,0.467783


In [181]:
# check how many times a certain option is sued
df_results['hiddens_dynamics_embedding'].value_counts()

hiddens_dynamics_embedding
[128]       4
[64]        4
[32]        3
[32, 32]    3
[64, 64]    2
not         2
Name: count, dtype: int64

In [182]:
df_results['hiddens_static_embedding'].value_counts()

hiddens_static_embedding
[64, 64]    8
[128]       3
[64]        3
[32]        2
[32, 32]    2
Name: count, dtype: int64

In [200]:
df_results['learning_rate'].value_counts()

learning_rate
0.0005    9
0.0001    3
5e-05     2
1e-05     2
0.001     2
Name: count, dtype: int64

In [187]:
# turn the columns hiddens_dynamics_embedding and hiddens_static_embedding into one-hot encoded columns
df_results_one_hot = pd.get_dummies(df_results, columns=['hiddens_dynamics_embedding', 'hiddens_static_embedding', 'learning_rate'])

df_results_one_hot

,max_avg_nse_value,hiddens_dynamics_embedding_[128],"hiddens_dynamics_embedding_[32, 32]",hiddens_dynamics_embedding_[32],"hiddens_dynamics_embedding_[64, 64]",hiddens_dynamics_embedding_[64],hiddens_dynamics_embedding_not,hiddens_static_embedding_[128],"hiddens_static_embedding_[32, 32]",hiddens_static_embedding_[32],"hiddens_static_embedding_[64, 64]",hiddens_static_embedding_[64],learning_rate_0.0001,learning_rate_0.0005,learning_rate_0.001,learning_rate_1e-05,learning_rate_5e-05
development_run_1803_153901,0.452472,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False
development_run_1903_083038,0.467783,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False
development_run_1903_105053,0.478346,False,False,False,True,False,False,False,False,False,True,False,False,True,False,False,False
development_run_1903_131019,0.491851,False,False,True,False,False,False,False,False,False,False,True,False,True,False,False,False
development_run_1903_153018,0.472811,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False
development_run_1903_175009,0.440265,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,True
development_run_1903_201011,0.369127,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True
development_run_2003_095615,0.460981,False,True,False,False,False,False,False,False,False,False,True,True,False,False,False,False
development_run_2003_122328,0.437849,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False
development_run_2003_145028,0.179698,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False


In [196]:
# crosscorrelate the max_avg_nse_value with the other columns
ccor_results = df_results_one_hot.corr()['max_avg_nse_value']

# aggregate the results per original non-one-hot encoded column
ccor_results_agg = ccor_results.groupby(lambda x: '_'.join(x.split('_')[:2])).max()

In [197]:
ccor_results_agg # highest correlation per column

hiddens_dynamics    0.306080
hiddens_static      0.469260
learning_rate       0.542291
max_avg             1.000000
Name: max_avg_nse_value, dtype: float64

In [198]:
ccor_results.sort_values(ascending=False) # highest correlation per column

max_avg_nse_value                      1.000000
learning_rate_0.0005                   0.542291
hiddens_static_embedding_[64, 64]      0.469260
hiddens_dynamics_embedding_[128]       0.306080
learning_rate_0.001                    0.170278
hiddens_dynamics_embedding_[64, 64]    0.143488
hiddens_static_embedding_[64]          0.079613
hiddens_dynamics_embedding_[32, 32]    0.060254
learning_rate_0.0001                  -0.002129
hiddens_static_embedding_[32]         -0.052564
hiddens_dynamics_embedding_[64]       -0.097738
learning_rate_5e-05                   -0.101960
hiddens_dynamics_embedding_not        -0.175937
hiddens_dynamics_embedding_[32]       -0.265305
hiddens_static_embedding_[128]        -0.347048
hiddens_static_embedding_[32, 32]     -0.372263
learning_rate_1e-05                   -0.928574
Name: max_avg_nse_value, dtype: float64

nu op basis van random forest feature importance, maar met 19 runs zijn dat er niet super veel, ik zou dus het minste waarde hechten aan deze resultaten.

In [199]:
# train a random forest model on X and y for the feature importance functionality
from sklearn.ensemble import RandomForestRegressor

X = df_results_one_hot.drop('max_avg_nse_value', axis=1)
y = df_results_one_hot['max_avg_nse_value']

model = RandomForestRegressor()
model.fit(X, y)

importances = model.feature_importances_


In [191]:
rf_feature_importances = pd.Series(importances, index=X.columns)

In [192]:
rf_feature_importances.sort_values(ascending=False)

learning_rate_1e-05                    0.793486
hiddens_dynamics_embedding_not         0.092321
hiddens_static_embedding_[32, 32]      0.022752
hiddens_dynamics_embedding_[64]        0.016445
hiddens_dynamics_embedding_[32, 32]    0.015026
learning_rate_0.0005                   0.012726
hiddens_static_embedding_[32]          0.009228
hiddens_dynamics_embedding_[128]       0.008541
hiddens_static_embedding_[128]         0.008146
learning_rate_5e-05                    0.007777
learning_rate_0.0001                   0.004640
hiddens_dynamics_embedding_[64, 64]    0.002901
hiddens_static_embedding_[64, 64]      0.001997
learning_rate_0.001                    0.001570
hiddens_dynamics_embedding_[32]        0.001361
hiddens_static_embedding_[64]          0.001083
dtype: float64

In [193]:
# sum the importances of each feature importance column, so we see how important these hyperparameters are according to our metric
rf_feature_importances.groupby(lambda x: '_'.join(x.split('_')[:2])).sum()

hiddens_dynamics    0.136595
hiddens_static      0.043206
learning_rate       0.820199
dtype: float64